In [ ]:
# verificação da versão do pytorch
!pip list | grep torch

In [ ]:
import os
from os.path import exists, join, basename, splitext
git_repo_url = 'https://github.com/lyuwenyu/RT-DETR.git'
project_name = splitext(basename(git_repo_url))[0]

In [ ]:


%cd /content
git_repo_url = 'https://github.com/lyuwenyu/RT-DETR.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  !pip install torch==2.0.1
  !pip install torchvision==0.15.2
  !pip install torchaudio==2.0.2
  !pip install onnx==1.14.0
  !pip install onnxruntime==1.15.1
  !pip install pycocotools
  !pip install PyYAML
  !pip install scipy


In [ ]:
print(project_name)

In [ ]:
!pip list | grep torch

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# verifica GPU
!nvidia-smi

In [ ]:
import os
import subprocess
from subprocess import Popen, PIPE

In [ ]:
# training on single-gpu

def treina_modelo(config_modelo,image_set):

    arquivo_saida = '/content/1_d_'+image_set+'.txt'
    comando_1 = f'touch {arquivo_saida}'
    comando_2 = f"""
    export CUDA_VISIBLE_DEVICES=0 &&
    python /content/drive/MyDrive/RT_DETR/RT-DETR/rtdetr_pytorch/tools/train.py -c {config_modelo} | tee {arquivo_saida}
    """
    print(comando_2)
    !$comando_1
    !$comando_2
    #process = subprocess.Popen(comando, stdout=PIPE, stderr=None, shell=True)
    #output = process.communicate()[0]
    #print(output)


In [ ]:
def obtem_lista_modelos(path_configs_modelos) -> list :
    lista_modelos = []
    files = os.listdir(path_configs_modelos)
    for file in files:
        if file.startswith('rtdetr') and file.endswith('.yml'):
            lista_modelos.append(file)
    return lista_modelos

In [ ]:
def copia_configs(configs_path,custom_configs_path,modelo,image_set):
    nome_modelo = modelo.split('.')[0]
    #print(nome_modelo)
    arquivo_origem = os.path.join(custom_configs_path,'optimizer.yml')
    arquivo_destino = os.path.join(configs_path,'rtdetr','include','optimizer.yml')
    comando = f'cp {arquivo_origem} {arquivo_destino}'
    #print(comando)
    subprocess.Popen(comando, shell=True)

    arquivo_origem = os.path.join(custom_configs_path,'coco_detection_'+image_set+'.yml')
    arquivo_destino = os.path.join(configs_path,'dataset','coco_detection.yml')
    comando = f'cp {arquivo_origem} {arquivo_destino}'
    #print(comando)
    subprocess.Popen(comando, shell=True)

    arquivo_origem = os.path.join(custom_configs_path,nome_modelo+'_'+image_set+'.yml')
    arquivo_destino = os.path.join(configs_path,'rtdetr',modelo)
    comando = f'cp {arquivo_origem} {arquivo_destino}'
    #print(comando)
    subprocess.Popen(comando, shell=True)

    return

In [ ]:
def exclui_checkpoints(modelo,image_set):
    nome_modelo = modelo.split('.')[0]+'_'+image_set
    rm_dir = os.path.join(os.sep,'content','output','1_d',nome_modelo)
    print(rm_dir)
    if os.path.exists(rm_dir):
        comando = f'rm -rf {rm_dir}'+'/*.pth'
        print(comando)
        !$comando
    return

In [ ]:
# import os
# rm_dir = '/content/output/1_d/rtdetr_r50vd_6x_coco_0'
# rm_dir = '/content/output/1_d/rtdetr_r50vd_6x_coco'
# print(rm_dir)
# if os.path.exists(rm_dir):
#     comando = f'rm -rf {rm_dir}'+'/*.pth'
#     !$comando
#     #print(comando)


In [ ]:
from google.colab import files
import shutil

def treina_modelos(project_path,lista_modelos,lista_image_set):
    configs_path = os.path.join(project_path,'configs')
    custom_configs_path = os.path.join(configs_path,'rtdetr','projeto_1_d')
    for modelo in lista_modelos:
        for image_set in lista_image_set:
            print('Início do treinamento do modelo ', modelo, ' para o conjunto de imagens ', image_set)
            copia_configs(configs_path=configs_path,custom_configs_path=custom_configs_path,modelo=modelo,image_set=image_set)
            config_modelo = os.path.join(configs_path,'rtdetr',modelo)
            treina_modelo(config_modelo,image_set)
            exclui_checkpoints(modelo,image_set)
            output_path = os.path.join(os.sep,'content','output','1_d','rtdetr_r50vd_6x_coco_'+image_set)
            #zip_file = os.path.join(output_path,'1_d_rtdetr_r50vd_6x_coco_'+image_set+'.zip')
            #origem_zip = os.path.join(output_path,'eval')
            #comando = '!zip -r '+zip_file+' '+origem_zip
            #print(comando)
            #!$comando

            origem_copia = os.path.join(output_path,'eval','best.pth')
            destino_copia = os.path.join('/content/drive/MyDrive/RT_DETR/RT-DETR','train','1_d',image_set)
            if os.path.isfile(origem_copia):
              if not os.path.exists(destino_copia):
                  os.makedirs(destino_copia)
              #files.download(origem_download)
              shutil.copy(origem_copia,destino_copia)

            origem_copia = os.path.join(output_path,'eval','latest.pth')
            if os.path.isfile(origem_copia):
              if not os.path.exists(destino_copia):
                  os.makedirs(destino_copia)
              #files.download(origem_download)
              shutil.copy(origem_copia,destino_copia)

            origem_copia = os.path.join(os.sep,'content','1_d_'+image_set+'.txt')
            if os.path.isfile(origem_copia):
              if not os.path.exists(destino_copia):
                  os.makedirs(destino_copia)
              #files.download(origem_download)
              shutil.copy(origem_copia,destino_copia)



In [ ]:
import sys
versao_modelo='rtdetr_pytorch'

project_path=os.path.join('/content/drive/MyDrive/RT_DETR/RT-DETR',versao_modelo)
sys.path.append(project_path)
path_config_modelos = os.path.join(project_path,'configs','rtdetr')
#print(path_config_modelos)
#lista_modelos = obtem_lista_modelos(path_config_modelos)
#lista_modelos = ['rtdetr_r50vd_6x_coco.yml','rtdetr_r34vd_6x_coco.yml']
lista_modelos = ['rtdetr_r50vd_6x_coco.yml']
#print(lista_modelos)
dataset_path = os.path.join(os.sep,'content','drive','MyDrive','RT_DETR','RT-DETR','dataset')
img_list_path = os.path.join(dataset_path,'img_list')
#img_list_path = '/content/drive/MyDrive/RT_DETR/dataset/img_list'
#lista_image_set = os.listdir(img_list_path)
lista_image_set = ['4']
lista_image_set.sort()
treina_modelos(project_path,lista_modelos,lista_image_set)


In [ ]:
#!zip -r /content/output/1_d/rtdetr_r50vd_6x_coco_2/1_d_rtdetr_r50vd_6x_coco_2.zip /content/output/1_d/rtdetr_r50vd_6x_coco_2/eval

In [ ]:
# !zip -r /content/output/1_d/rtdetr_r50vd_6x_coco_0/1_d_rtdetr_r50vd_6x_coco_0.zip /content/output/1_d/rtdetr_r50vd_6x_coco_0/eval
# !zip -r /content/output/1_d/rtdetr_r50vd_6x_coco_1/1_d_rtdetr_r50vd_6x_coco_1.zip /content/output/1_d/rtdetr_r50vd_6x_coco_1/eval
# !zip -r /content/output/1_d/rtdetr_r50vd_6x_coco_2/1_d_rtdetr_r50vd_6x_coco_2.zip /content/output/1_d/rtdetr_r50vd_6x_coco_2/eval
# !zip -r /content/output/1_d/rtdetr_r50vd_6x_coco_3/1_d_rtdetr_r50vd_6x_coco_3.zip /content/output/1_d/rtdetr_r50vd_6x_coco_3/eval
# !zip -r /content/output/1_d/rtdetr_r50vd_6x_coco_4/1_d_rtdetr_r50vd_6x_coco_4.zip /content/output/1_d/rtdetr_r50vd_6x_coco_4/eval

# # !zip -r /content/output/1_d/rtdetr_r50vd_6x_coco_0/eval
# # !zip -r /content/output/1_d/rtdetr_r50vd_6x_coco_1/eval
# # !zip -r /content/output/1_d/rtdetr_r50vd_6x_coco_2/eval
# # !zip -r /content/output/1_d/rtdetr_r50vd_6x_coco_3/eval
# # !zip -r /content/output/1_d/rtdetr_r50vd_6x_coco_4/eval

# from google.colab import files
# files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_0/1_d_rtdetr_r50vd_6x_coco_0.zip")
# files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_1/1_d_rtdetr_r50vd_6x_coco_1.zip")
# files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_2/1_d_rtdetr_r50vd_6x_coco_2.zip")
# files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_3/1_d_rtdetr_r50vd_6x_coco_3.zip")
# files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_4/1_d_rtdetr_r50vd_6x_coco_4.zip")

# # files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_0/eval.zip")
# # files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_0/eval.zip")
# # files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_0/eval.zip")
# # files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_0/eval.zip")
# # files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_0/eval.zip")

# files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_0/log.txt")
# files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_1/log.txt")
# files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_2/log.txt")
# files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_3/log.txt")
# files.download("/content/output/1_d/rtdetr_r50vd_6x_coco_4/log.txt")

# files.download("/content/output/1_d.txt")